# ODE Review: Projectile Motion

Let's try solving for projectile motion with drag (air resistance).

We want to solve:

\begin{align*}
\dot{\bf r} &= {\bf v} \\
\dot{\bf v} &= \frac{1}{m} {\bf F}_a({\bf v}) - g \hat{\bf y}
\end{align*}

where $m$ is the mass of the projectile and

$${\bf F}_a({\bf v}) = - \frac{1}{2}C \rho_\mathrm{air} A |v| {\bf v}$$

is the [Newton drag term](https://en.wikipedia.org/wiki/Projectile_motion#Trajectory_of_a_projectile_with_Newton_drag) that
is applicable for large Reynolds numbers (turbulent), and $\rho_\mathrm{air}$ is the density of air, $C$ is the drag coefficient, $A$ is the cross-section
area of the projectile.

```{note}
Since the force here depends on velocity, we can't use the velocity-Verlet method.
```

We'll consider a baseball.  Then we can take:

* $C = 0.3$
* $A = \pi (d/2)^2$ with the diameter of a baseball, $d = 7.4~\mathrm{cm}$
* $m = 145~\mathrm{g}$
* $\rho_\mathrm{air} = 1.2\times 10^{-3}~\mathrm{g/cm^3}$

We also take gravity as constant, $g = 981~\mathrm{cm/s^2}$.

We'll imagine throwing the baseball from some height $y_0$ above the ground ($y = 0$) at an angle $\theta$ from the horizontal with velocity
magnitude $V$.

This means our initial conditions are:

\begin{align*}
x_0 &= 0 \\
y_0 &= y_0 \\
u_0 &= V \cos(\theta) \\
v_0 &= V \sin(\theta)
\end{align*}

and we want to integrate until the ball hits the ground, $y(t_\mathrm{max}) = 0$.

Let's implement this using 4th-order Runge-Kutta.